# Análise comparativa de modelos

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, Markdown as md
%matplotlib inline

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer

In [2]:
df = pd.read_csv("../data/raw/Churn_Modelling.csv")
df_dict = pd.read_csv("../data/external/dicionario.csv")

#Tratamento Quantitativo
var_quantitativa = df_dict.query("Tipo == 'Quantitativo'").Variavel.to_list()
# Tratamento Qualitativo
coluna_predicao = 'Exited'
var_nominal= df_dict.query("Subtipo == 'Nominal' & Variavel != @coluna_predicao").Variavel.to_list()

df_non_predicao = df.drop(columns=[coluna_predicao], axis=1)

## 1 - Preparação de dados

In [3]:
#Dataset
display(md("### `DataSet`"))
display(df.head(10))

#Print Dicionario
display(md("### `Dicionário de Dados`"))
display(df_dict)


dado_faltante = df.isnull().any(axis=1).sum()
if dado_faltante != 0:
    display(md("---"))
    display(md(f"Existem `{dado_faltante}` dados faltantes no Dataset."))
    display(md("---"))
else:
    print("Não existe dado faltante no Dataset.")

### `DataSet`

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2,134603.88,1,1,1,71725.73,0


### `Dicionário de Dados`

,Variavel,Descrição,Tipo,Subtipo
0,RowNumber,O número sequencial atribuído a cada linha no ...,Quantitativo,Discreta
1,CustomerId,Identificador único de cada cliente,Quantitativo,Discreta
2,Surname,Indica o sobrenome do cliente.,Qualitativo,Nominal
3,CreditScore,Indica a pontuação de crédito do cliente.,Quantitativo,Discreta
4,Geography,Localização geográfica do cliente (país ou reg...,Qualitativo,Nominal
5,Gender,Gênero do cliente,Qualitativo,Nominal
6,Age,Idade do cliente,Quantitativo,Discreta
7,Tenure,Indica número de anos que o cliente está no ba...,Quantitativo,Discreta
8,Balance,Indica o saldo da conta do cliente,Quantitativo,Contínua
9,NumOfProducts,Indica o número de produtos bancários que o cl...,Quantitativo,Discreta


Não existe dado faltante no Dataset.


### `Função para reconhecimento de outliers`

---
`Função outliers:` Qualquer valor menor que o limite inferior será substituído por **lower_bound** e qualquer valor maior que o limite superior será substituído por **upper_bound**

---

In [4]:
# Função para tratamento de outliers usando IQR
def outliers(X):
    Q1 = np.quantile(X, 0.25)
    Q3 = np.quantile(X, 0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return np.clip(X, lower_bound, upper_bound)

### `Tratamento de dados`

In [5]:
# Pipeline para variáveis quantitativas
quantitativo_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),  # Tratamento de dados faltantes
    ('outlier', FunctionTransformer(outliers, validate=False)),  # Tratamento de outliers
    ('normalization', StandardScaler())  # Normalização
])

# Pipeline para variáveis qualitativas
qualitativo_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Tratamento de dados faltantes
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  # Codificação de variáveis
])

# Combinando pipelines com ColumnTransformer (junta as transformações e aplica de uma vez)
preprocessor = ColumnTransformer([
    ('quantitative', quantitativo_pipeline, var_quantitativa),
    ('qualitative', qualitativo_pipeline, var_nominal)
])


# Aplicando o pipeline ao DataFrame
#preprocessor.fit(df_non_predicao)
